In [1]:
import earthaccess
import xarray as xr

In [2]:
auth = earthaccess.login(persist=True)

In [13]:
# Level 2 data
results = earthaccess.search_data(
    short_name = 'PACE_OCI_L2_LANDVI',
    temporal = ("2024-03-05", "2024-03-31")
)
len(results)

2492

In [14]:
# Level 2 data March 2024
# bounding_box = (lat_min, lon_min, lat_max, lon_max)
results = earthaccess.search_data(
    short_name = 'PACE_OCI_L2_LANDVI',
    temporal = ("2024-03-05", "2024-03-31"),
    bounding_box = (3.9600833333, 2.8231658333, 6.6408783333, 8.692175)
)
len(results)

25

In [15]:
# Step 1 set up the fileset
fileset = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/25 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/25 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/25 [00:00<?, ?it/s]

In [16]:
# Step 2 open L2 swath files with xarray open_datatree
datatree = xr.open_datatree(fileset[1], decode_timedelta=False) 
datatree.groups

('/',
 '/sensor_band_parameters',
 '/scan_line_attributes',
 '/geophysical_data',
 '/navigation_data',
 '/processing_control',
 '/processing_control/input_parameters',
 '/processing_control/flag_percentages')

In [17]:
# Step 3 merge the groups all together.
# This works due to the values in each group being the same shape.
ds_l2 = xr.merge(datatree.to_dict().values())
ds_l2 = ds_l2.set_coords(("longitude", "latitude"))
ds_l2

<xarray.Dataset> Size: 113MB
Dimensions:      (number_of_bands: 286, number_of_reflective_bands: 286,
                  number_of_lines: 1710, pixels_per_line: 1272)
Coordinates:
    longitude    (number_of_lines, pixels_per_line) float32 9MB ...
    latitude     (number_of_lines, pixels_per_line) float32 9MB ...
Dimensions without coordinates: number_of_bands, number_of_reflective_bands,
                                number_of_lines, pixels_per_line
Data variables: (12/34)
    wavelength   (number_of_bands) float64 2kB ...
    vcal_gain    (number_of_reflective_bands) float32 1kB ...
    vcal_offset  (number_of_reflective_bands) float32 1kB ...
    F0           (number_of_reflective_bands) float32 1kB ...
    aw           (number_of_reflective_bands) float32 1kB ...
    bbw          (number_of_reflective_bands) float32 1kB ...
    ...           ...
    pri          (number_of_lines, pixels_per_line) float32 9MB ...
    cire         (number_of_lines, pixels_per_line) float32 9MB ...
    car          (number_of_lines, pixels_per_line) float32 9MB ...
    mari         (number_of_lines, pixels_per_line) float32 9MB ...
    l2_flags     (number_of_lines, pixels_per_line) int32 9MB ...
    tilt         (number_of_lines) float32 7kB ...
Attributes: (12/47)
    title:                             OCI Level-2 Data LANDVI
    product_name:                      PACE_OCI.20240306T120228.L2.LANDVI.V3_...
    processing_version:                3.1
    history:                           l2gen par=/data11/sdpsoper/vdc/vpu30/w...
    instrument:                        OCI
    platform:                          PACE
    ...                                ...
    geospatial_lon_min:                0.00011603648
    startDirection:                    Ascending
    endDirection:                      Ascending
    day_night_flag:                    Day
    earth_sun_distance_correction:     1.0157673358917236
    geospatial_bounds:                 POLYGON ((27.11761 -2.71841, 3.37181 -...

In [18]:
ds_l2_size_gb = ds_l2.nbytes / 1e9
print(f"Dataset size: {ds_l2_size_gb:.2f} GB")

Dataset size: 0.11 GB


In [20]:
ds_l2.to_netcdf("l2_2024_03.nc")

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 15-17: surrogates not allowed